IMPORTS

In [65]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

BASES

In [66]:
# LIEN ACTEURS

# df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
# df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
# df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

MERGES

In [67]:
# CHANGEMENTS DANS BASE TITLE EPISODES

df_title_episodes = pd.read_csv('BD/title.episode.tsv.gz', compression = 'gzip', sep = '\t')

df_title_episodes[['seasonNumber', 'episodeNumber']] = df_title_episodes[['seasonNumber', 'episodeNumber']].replace('\\N', 0)
df_title_episodes[['seasonNumber', 'episodeNumber']] = df_title_episodes[['seasonNumber', 'episodeNumber']].apply(pd.to_numeric)

df_serie = df_title_episodes.groupby('parentTconst').sum()
df_serie = df_serie[['seasonNumber', 'episodeNumber']]
df_serie = df_serie.reset_index()

df_episodes = pd.DataFrame(df_title_episodes['tconst'])

df_serie['titleType'] = 'serie'
df_serie = df_serie.rename(columns = {'parentTconst': 'tconst'})
df_episodes['titleType'] = 'episode'
df_serie = df_serie.rename(columns = {'parentTconst': 'tconst'})
df_title_episodes = pd.concat([df_serie, df_episodes])

df_serie, df_episodes = 0, 0

In [68]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_9447/3795572048.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [69]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

df_title_ratings, df_title_basics = 0, 0

In [70]:
df2 = pd.merge(df1,
               df_title_episodes,
               how = 'outer',
               on = 'tconst')

df1, df_title_episodes = 0, 0

In [71]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

In [72]:
df_3 = pd.merge(df2,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

df2, df_title_akas = 0, 0

In [73]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_9447/1088705580.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


In [74]:
df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

df_tmbd_full, df_3 = 0, 0

CLEAN DU MERGE

In [75]:
# DROP CE QUI N'EST PAS FILM OU SERIE


df_films = df_films[(df_films['titleType_x'] == 'movie') | (df_films['titleType_x'] == 'tvEpisode') | (df_films['titleType_x'] == 'tvSeries') | (df_films['titleType_y'] == 'serie') | (df_films['titleType_y'] == 'episode')]

In [76]:
df_films['runtimeMinutes'][(df_films['titleType_x'] == 'tvEpisode') | (df_films['titleType_x'] == 'tvSeries') | (df_films['titleType_y'] == 'serie') | (df_films['titleType_y'] == 'episode')] = 0

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_9447/3967036656.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_films['runtimeMinutes'][(df_films['titleType_x'] == 'tvEpisode') | (df_films['titleType_x'] == 'tvSeries') | (d

In [77]:
print(len(df_films))

46732493


In [78]:
# COLONNES NUMERIQUES

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)
df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

In [80]:

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [81]:
# On transforme les types de données

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year
df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [82]:
df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0's"
df_films['Decennie'] = df_films['Decennie'].replace("0.00's", None)
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])


RUNTIME

In [83]:
df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

: 

In [88]:
df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [91]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [93]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [94]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [96]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [97]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [98]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [99]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [100]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [101]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [102]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [103]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [104]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [106]:
df_films = pd.concat([df_final, df_null])

In [107]:
print(len(df_films))

3395528


In [108]:
df_final, df_null = 0, 0

In [109]:
df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [111]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [112]:
df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [113]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [114]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [115]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [116]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [117]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/612399471.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/612399471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler

In [118]:
df_a_garder_4 = df_a_travailler_3

In [119]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_5354/36534397

In [120]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [121]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [122]:
df_films = pd.concat([df_final, df_null9])

In [124]:
df_final, df_null9 = 0, 0

In [125]:
# GENRE

df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [ ]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

In [ ]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x
  
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [ ]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [ ]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [ ]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [ ]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)
# df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

In [ ]:
df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)

In [ ]:
# TITLE

df_films['title'] = df_films['original_title'].fillna(df_films['title_y']).fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['originalTitle'])
df_films = df_films.drop(['original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)

In [ ]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']
df_films = df_films.drop(['isAdult'], axis=1)

EXPORT

In [ ]:
df_films.to_csv('P2_G5_films_serie.csv.gz', index=False, compression='gzip')